In [ ]:
from backend.TextPreprocessingUtils import vectorize_text_data
train, test = vectorize_text_data("data/raw_data.csv")
test

In [ ]:
for x in range(test.shape[1]):
    tfidf_score = test.iat[5,x] # For the every column (word in vocabulary) in first tweet (row 0)
    if (tfidf_score > 0.0):
        label = test.columns[x]
        print("TF-IDF Score for word " + label + " is: " + str(tfidf_score))

had to download the below for textpreprocessingutils.py to work, thought it might be useful for others - joe

In [ ]:
df = test

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from sklearn.decomposition import TruncatedSVD
import numpy as np
import pandas as pd
import joblib
from backend.TextPreprocessingUtils import vectorize_text_data

df_train, df_test, label_train, label_test = vectorize_text_data('data/raw_data.csv')

# df_list = []

# for chunk in chunks:
#     Identify columns that are float64
#     float_cols = chunk.select_dtypes(include=['float64']).columns
     # Convert those columns to float32
#     chunk[float_cols] = chunk[float_cols].astype("float32")
#     df_list.append(chunk)

# df = pd.concat(df_list, ignore_index=True)


Loading data...
Preprocessing text data...
Vectorizing text data with TF-IDF...
TF-IDF Vectorizer saved as 'tfidf_vectorizer.pkl'.
Splitting data into training and testing sets...
TF-IDF vectorization completed, and data saved successfully!


In [ ]:
# Pop out the column with value "label"
label_column = df.pop("label")

# Insert it back at index 0
df.insert(0, "label", label_column)

df


In [3]:
for x in range(df_train.shape[1]):
    tfidf_score = df_train.iat[5,x] # For the every column (word in vocabulary) in first tweet (row 0)
    if (tfidf_score > 0.0):
        label = df_train.columns[x]
        print("TF-IDF Score for word " + label + " is: " + str(tfidf_score))

TF-IDF Score for word http is: 1.0


In [4]:
# X = df.drop('label', axis=1).values

# Normalize the features so they have a mean of 0 and std of 1
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# Reshape X to be [samples, timesteps, features]
xtrain = df_train.to_numpy()
xtest = df_test.to_numpy()
ytrain = label_train.to_numpy()
ytest = label_test.to_numpy()

# xtrain = xtrain.reshape((xtrain.shape[0], xtrain.shape[1], 1))
# xtest = xtest.reshape((xtest.shape[0], xtest.shape[1], 1))



xtrain.shape

(576000, 100)

In [ ]:
# Separate labels and features
# y = df['label'].values
# X = df.drop('label', axis=1).values

svd = TruncatedSVD(n_components=100)
xtrain_reduced = svd.fit_transform(xtrain)
xtest_reduced = svd.transform(xtest)

joblib.dump(svd, 'svd.pkl')
training_amount = int(len(xtrain) * 0.01)

testing_x = xtest_reduced[-training_amount:]
testing_y = ytest[-training_amount:]

print("formatted X and y data")

formatted X and y data


In [5]:
# Compute class weights to help counter class imbalance
classes = np.unique(ytrain)
class_weights = compute_class_weight('balanced', classes=classes, y=ytrain)
class_weight_dict = {int(cls): weight for cls, weight in zip(classes, class_weights)}
print("Computed class weights:", class_weight_dict)

Computed class weights: {0: np.float64(0.5996951549628938), 1: np.float64(3.0076444295918794)}


In [7]:
model = Sequential([
    Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(xtrain.shape[1], 1)),  # First layer
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(xtrain.shape[1], 1)),  # First layer
    # Conv1D(filters=32, kernel_size=5, activation='relu'),  # Second layer
    # Conv1D(filters=32, kernel_size=5, activation='relu'),  # Third layer
    Conv1D(filters=8, kernel_size=5, activation='relu'),   # Fourth layer
    Flatten(),
    Dense(1, activation='sigmoid')  # Output: Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with class weights
model.fit(xtrain, ytrain, epochs=10, batch_size=640, class_weight=class_weight_dict)

c:\Users\mrjoe\OneDrive\Desktop\New folder\sweng25_group11-microsoftspamdetection\sklearn-env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 145s 159ms/step - accuracy: 0.6795 - loss: 0.5935
Epoch 2/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 152s 169ms/step - accuracy: 0.7140 - loss: 0.5709
Epoch 3/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 159s 177ms/step - accuracy: 0.7124 - loss: 0.5696
Epoch 4/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 142s 157ms/step - accuracy: 0.7120 - loss: 0.5697
Epoch 5/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 439s 488ms/step - accuracy: 0.7154 - loss: 0.5688
Epoch 6/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 117s 130ms/step - accuracy: 0.7149 - loss: 0.5687
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 140s 155ms/step - accuracy: 0.7161 - loss: 0.5682
Epoch 8/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 158s 173ms/step - accuracy: 0.7157 - loss: 0.5672
Epoch 9/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 138s 154ms/step - accuracy: 0.7156 - loss: 0.5685
Epoch 10/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 130s 145ms/step - accuracy: 0.7158 - loss: 0.5664


In [8]:
import joblib

joblib.dump(model, 'cnn.pkl')

['cnn.pkl']

In [ ]:
success_count = 0
num_tests = 500

for i in range(num_tests):
    # Choose a sample index from your dataset (you can change this index as needed)
    sample_index = np.random.randint(0, testing_x.shape[0])
    sample_input = testing_x[sample_index].reshape(1, testing_x.shape[1])  # shape (1, timesteps)

    # Get the model's predicted probability for the sample being spam
    predicted_probability = model.predict(sample_input, verbose=0)

    # Retrieve the actual label from your labels array
    actual_label = testing_y[sample_index]

    predicted_label = round(predicted_probability)

    percentage_success = success_count / (i + 1) * 100
    print(f"{i=} {percentage_success:.2f}% o={predicted_probability:.4f} p={predicted_label} a={actual_label}         ", end="\r")
    success_count += predicted_label == actual_label

print(f"{success_count=} success! (thats like {(success_count / num_tests * 100):.4f}%)    ")

In [44]:
# Save the model in the keras format
model.save("spam_detection_model_CNN.keras")

# Alternatively, to save in the TensorFlow SavedModel format:
# model.save("spam_detection_model")


In [45]:
import numpy as np

success_count = 0
for i in range(100):
    # Choose a sample index from your dataset (you can change this index as needed)
    sample_index = np.random.randint(0, xtest.shape[0])
    sample_input = xtest_reduced[sample_index].reshape(1, xtest_reduced.shape[1], 1)  # shape (1, timesteps, features)

    # Get the model's predicted probability for the sample being spam
    predicted_probability = model.predict(sample_input, verbose=0)[0][0]

    # Retrieve the actual label from your labels array
    actual_label = y[sample_index]

    predicted_label = round(predicted_probability)
    success_count += predicted_label == actual_label

success_count

np.int64(58)

_______________________________________________________
The below LSTM didn't work - keeping here for the moment for future reference

In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import numpy as np

# Separate labels and features
y = df['label'].values
X = df.drop('label', axis=1).values

# Normalize the features so they have a mean of 0 and std of 1
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape X to be [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Compute class weights to address class imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Computed class weights:", class_weight_dict)

# Define a more complex LSTM model architecture
model = Sequential()
# First LSTM layer with return_sequences=True to stack another LSTM
model.add(LSTM(64, input_shape=(X.shape[1], 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Second LSTM layer (no return_sequences needed for final output)
model.add(LSTM(32))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Final dense layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with class weights
model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2, class_weight=class_weight_dict)



In [ ]:
import joblib

svm_model = joblib.load("svm.pkl")
print("Model loaded successfully with joblib!")